# CIA World Factbook Project

The CIA Factbook contains various statistics about all of the countries in the world, including:

 - population — the global population.
 - population_growth — the annual population growth rate, as a percentage.
 - area — the total land and water area.

In [1]:
!pip install ipython-sql

You should consider upgrading via the 'c:\users\florence\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [3]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [4]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The table contains 8 columns; these 8 columns are:

 - name - name of the country
 - area - total land and water area of the country
 - area_land - land area of the country in square kilometres
 - area_water - water area of the country in square kilometres
 - population - population of the country
 - population_growth - how quickly the population of the country grows as a      percentage
 - birth_rate - number of births per 1,000 people 
 - death_rate - number of deaths per 1,000 people
 - migration_rate

In [5]:
%%sql
SELECT MIN(population),
       MAX(population),
       MIN(population_growth),
       MAX(population_growth)
  FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


The minimum population is zero, and the maximum population is approximately 7.2 billion, which is around the world's total population. The maximum popluation growth is 4.02%.

In [6]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [7]:
%%sql
SELECT *
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


We need to exclude Antartica and the world from our analysis of the minimum and maximum population, as Antartica only has research scientists there, many of whom do not live there permanently, and the world row is for the entire world, which includes all of the countries on the list.

In [8]:
%%sql
SELECT MIN(population),
       MAX(population),
       MIN(population_growth),
       MAX(population_growth)
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [9]:
%%sql
SELECT AVG(population) AS avg_pop, AVG(area) AS avg_area
  FROM facts
 WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


In [10]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <>'World' AND name <> 'Antartica')
 AND area <  (SELECT AVG(area)
                FROM facts
               WHERE name <> 'World' AND name <> 'Antartica');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Some of these countries are known for being densely populated, however I'm surprised that neither China nor India is on this list, as they're both known for their large populations and being densely populated.

In [11]:
%%sql

SELECT name, population
  FROM facts
 WHERE name <> 'World' AND population == (SELECT MAX(population)
                                            FROM facts
                                            WHERE name <> 'World');

 * sqlite:///factbook.db
Done.


name,population
China,1367485388


In [12]:
%%sql

SELECT name, population_growth
  FROM facts
 WHERE name <> 'World' AND population_growth == (SELECT MAX(population_growth)
                                            FROM facts
                                            WHERE name <> 'World');

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02


## Countries with highest water to land ratio

In [13]:
%%sql

SELECT name, CAST(area_water AS FLOAT) / area_land AS water_land_ratio
  FROM facts
 WHERE name <> 'World' AND name <> 'Antartica'
 ORDER BY water_land_ratio DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


name,water_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


The highest average water area to land area ratios belong to relatively small islands; Malawi is the country with the highest water to land ratio that isn't an island or group of islands. Only 2 countries/territories contain more water than land.

In [14]:
%%sql

SELECT COUNT(*)
  FROM facts
 WHERE name <> 'World' AND name <> 'Antartica';

 * sqlite:///factbook.db
Done.


COUNT(*)
260


## Countries with the highest population density

In [15]:
%%sql

SELECT name, CAST(population AS FLOAT) / area AS pop_density
  FROM facts
 WHERE name <> 'World' AND name <> 'Antartica'
 ORDER BY pop_density DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


name,pop_density
Macau,21168.964285714286
Monaco,15267.5
Singapore,8141.279770444763
Hong Kong,6445.041516245487
Gaza Strip,5191.819444444444
Gibraltar,4876.333333333333
Bahrain,1771.8592105263158
Maldives,1319.6409395973155
Malta,1310.01582278481
Bermuda,1299.9259259259259


This list is quite different to the list we got previously, and mostly consists of very small countries.

## Countries that have a higher death rate than birth rate

In [16]:
%%sql

SELECT name, death_rate, birth_rate
  FROM facts
 WHERE death_rate > birth_rate;

 * sqlite:///factbook.db
Done.


name,death_rate,birth_rate
Austria,9.42,9.41
Belarus,13.36,10.7
Bosnia and Herzegovina,9.75,8.87
Bulgaria,14.44,8.92
Croatia,12.18,9.45
Czech Republic,10.34,9.63
Estonia,12.4,10.51
Germany,11.42,8.47
Greece,11.09,8.66
Hungary,12.73,9.16


In [17]:
%%sql

SELECT name, death_rate, birth_rate, CAST(death_rate AS FLOAT) / birth_rate AS death_birth_ratio
  FROM facts
 WHERE death_rate > birth_rate AND name <> 'World'
ORDER BY death_birth_ratio DESC;

 * sqlite:///factbook.db
Done.


name,death_rate,birth_rate,death_birth_ratio
Bulgaria,14.44,8.92,1.6188340807174888
Serbia,13.66,9.08,1.5044052863436124
Latvia,14.31,10.0,1.431
Lithuania,14.27,10.1,1.4128712871287128
Hungary,12.73,9.16,1.3897379912663756
Monaco,9.24,6.65,1.3894736842105262
Slovenia,11.37,8.42,1.350356294536817
Ukraine,14.46,10.72,1.3488805970149254
Germany,11.42,8.47,1.3482880755608027
Saint Pierre and Miquelon,9.72,7.42,1.3099730458221026


Bulgaria has the highest death rate, and the countries that have the highest death to birth rate ratios are mostly based in Europe.

In [18]:
%%sql

SELECT name, death_rate, birth_rate, CAST(death_rate AS FLOAT) / birth_rate AS death_birth_ratio
  FROM facts
 WHERE name <> 'World'
ORDER BY death_birth_ratio;

 * sqlite:///factbook.db
Done.


name,death_rate,birth_rate,death_birth_ratio
Kosovo,None,None,None
Holy See (Vatican City),None,None,None
Ashmore and Cartier Islands,None,None,None
Christmas Island,None,None,None
Cocos (Keeling) Islands,None,None,None
Coral Sea Islands,None,None,None
Heard Island and McDonald Islands,None,None,None
Norfolk Island,None,None,None
Clipperton Island,None,None,None
French Southern and Antarctic Lands,None,None,None


The country/territory with the lowest death to birth ratio is the Gaza Strip. 

## Countries that will add the most people to their populations

In [19]:
%%sql

SELECT *, (population*(population_growth/100)) AS pop_growth
  FROM facts
 WHERE name <> 'World'
ORDER BY pop_growth DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,pop_growth
77,in,India,3287263,2973193,314070,1251695584,1.22,19.55,7.32,0.04,15270686.124799998
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44,6153684.246
129,ni,Nigeria,923768,910768,13000,181562056,2.45,37.64,12.9,0.22,4448270.372
132,pk,Pakistan,796095,770875,25220,199085847,1.46,22.58,6.49,1.54,2906653.3662
58,et,Ethiopia,1104300,None,104300,99465819,2.89,37.27,8.19,0.22,2874562.1691
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46,2703323.92
186,us,United States,9826675,9161966,664709,321368864,0.78,12.49,8.15,3.86,2506677.1392
78,id,Indonesia,1904569,1811569,93000,255993674,0.92,16.72,6.37,1.16,2355141.8008
40,cg,"Congo, Democratic Republic of the",2344858,2267048,77810,79375136,2.45,34.88,10.07,0.27,1944690.8320000002
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09,1626073.8536


India is due to add the most people to their population, followed by China.

## Future questions?